In [1]:
# set up libraries
    # We will need the RBCPath type from the rbclib package to load data from the RBC.
from rbclib import RBCPath
    # We'll also want to load some data directly from the filesystem.
from pathlib import Path
    # We'll want to load/process some of the data using pandas and numpy.
import pandas as pd
import numpy as np

In [2]:
# functions form the given analysis function 
def load_fsdata(participant_id, local_cache_dir=(Path.home() / 'cache')):
    "Loads and returns the dataframe of a PNC participant's FreeSurfer data."
    # from Neurohakademy 2025 organizers

    # Check that the local_cache_dir exists and make it if it doesn't.
    if local_cache_dir is not None:
        local_cache_dir = Path(local_cache_dir)
        local_cache_dir.mkdir(exist_ok=True)
    
    # Make the RBCPath and find the appropriate file:
    pnc_freesurfer_path = RBCPath(
        'rbc://PNC_FreeSurfer/freesurfer',
        # We provide the local_cache_dir to the RBCPath object; all paths made
        # from this object will use the same cache directory.
        local_cache_dir=local_cache_dir)
    participant_path = pnc_freesurfer_path / f'sub-{participant_id}'
    tsv_path = participant_path / f'sub-{participant_id}_regionsurfacestats.tsv'

    # Use pandas to read in the TSV file:
    with tsv_path.open('r') as f:
        data = pd.read_csv(f, sep='\t')

    # Return the loaded data:
    return data

In [3]:
# My functions 
def flatten_RBC_participant(participant_id):
    "Flatten the data from RBC for a given participant."
    
    # Load Data
    df = load_fsdata(participant_id)
    
    # Summarize Feature Info
    df['feature_info'] = df[['atlas', 'hemisphere', 'StructName']].agg('__'.join, axis=1)
    df = df.drop(columns=['atlas', 'hemisphere', 'StructName'])
    
    # Make a place to store the data
    output_df = pd.DataFrame(columns = ['measure_info', 'value'])
    
    # Iterate through the measures flattening them and adding them to the new df
    allowed_measures = ['NumVert', 'SurfArea', 'GrayVol', 'ThickAvg', 'ThickStd', 'MeanCurv', 'GausCurv', 'FoldInd', 'CurvInd', 
                        'Index', 'SegId', 'Mean_wgpct', 'StdDev_wgpct', 'Min_wgpct', 'Max_wgpct', 'Range_wgpct', 'SNR_wgpct', 
                        'Mean_piallgi', 'StdDev_piallgi', 'Min_piallgi', 'Max_piallgi', 'Range_piallgi']
    for measure in allowed_measures:
        temp_df = pd.DataFrame(columns = ['measure_info', 'value'])
        labels = df['feature_info'].apply(lambda x : f'{x}__{measure}')
        values = df[measure]
        temp_df['measure_info'] = labels
        temp_df['value'] = values
        output_df = pd.concat([output_df, temp_df])
    
    return output_df


def make_RBC_data_into_a_table(participant_id_list):
    '''Make a dataframe of all RBC participants in the given list'''

    # Initialize the df with the first participant
    output_flattened_df = flatten_RBC_participant(participant_id_list[0])
    output_flattened_df = output_flattened_df.rename(columns={'value':participant_id_list[0]}).set_index('measure_info').T

    # add in all the other participants
    for participant_id in participant_id_list[1:]:
        temp_df = flatten_RBC_participant(participant_id)
        temp_df = temp_df.rename(columns={'value':participant_id}).set_index('measure_info').T
        output_flattened_df = pd.concat([output_flattened_df, temp_df])
    
    return output_flattened_df

In [4]:
example_participant_id = 1000393599

# Load Data
example_df = load_fsdata(example_participant_id)

# Remove session id because it is empty
example_df = example_df.drop(columns=['session_id'])

# Summarize Feature Info
example_df['feature_info'] = example_df[['atlas', 'hemisphere', 'StructName']].agg('__'.join, axis=1)
# example_df = example_df.drop(columns=['atlas', 'hemisphere', 'StructName'])

# Make a place to store the data
output_df = pd.DataFrame(columns = ['measure_info', 'value'])

# Iterate through the measures flattening them and adding them to the new df
allowed_measures = ['NumVert', 'SurfArea', 'GrayVol', 'ThickAvg', 'ThickStd', 'MeanCurv', 'GausCurv', 'FoldInd', 'CurvInd', 
                    'Index', 'SegId', 'Mean_wgpct', 'StdDev_wgpct', 'Min_wgpct', 'Max_wgpct', 'Range_wgpct', 'SNR_wgpct', 
                    'Mean_piallgi', 'StdDev_piallgi', 'Min_piallgi', 'Max_piallgi', 'Range_piallgi']
for measure in allowed_measures:
    temp_df = pd.DataFrame(columns = ['measure_info', 'value'])
    labels = example_df['feature_info'].apply(lambda x : f'{x}__{measure}')
    values = example_df[measure]
    temp_df['measure_info'] = labels
    temp_df['value'] = values
    output_df = pd.concat([output_df, temp_df])
    
output_df

,measure_info,value
0,aparc.DKTatlas__lh__caudalanteriorcingulate__N...,1668
1,aparc.DKTatlas__lh__caudalmiddlefrontal__NumVert,3308
2,aparc.DKTatlas__lh__cuneus__NumVert,4102
3,aparc.DKTatlas__lh__entorhinal__NumVert,737
4,aparc.DKTatlas__lh__fusiform__NumVert,4115
...,...,...
13735,Yeo2011_7Networks_N1000__rh__7Networks_3__Rang...,2.25
13736,Yeo2011_7Networks_N1000__rh__7Networks_4__Rang...,3.2703
13737,Yeo2011_7Networks_N1000__rh__7Networks_5__Rang...,2.7538
13738,Yeo2011_7Networks_N1000__rh__7Networks_6__Rang...,3.055


In [5]:
example_df

,subject_id,atlas,hemisphere,StructName,NumVert,SurfArea,GrayVol,ThickAvg,ThickStd,MeanCurv,...,Min_wgpct,Max_wgpct,Range_wgpct,SNR_wgpct,Mean_piallgi,StdDev_piallgi,Min_piallgi,Max_piallgi,Range_piallgi,feature_info
0,sub-1000393599,aparc.DKTatlas,lh,caudalanteriorcingulate,1668,1121,3493,2.870,0.588,0.122,...,-1.8413,42.8855,44.7269,4.4281,1.9877,0.0777,1.8054,2.1455,0.3402,aparc.DKTatlas__lh__caudalanteriorcingulate
1,sub-1000393599,aparc.DKTatlas,lh,caudalmiddlefrontal,3308,2236,7030,2.882,0.537,0.109,...,7.1531,40.4774,33.3243,5.0341,3.3898,0.2448,2.7003,3.8032,1.1029,aparc.DKTatlas__lh__caudalmiddlefrontal
2,sub-1000393599,aparc.DKTatlas,lh,cuneus,4102,2619,5753,2.019,0.490,0.125,...,-13.1617,33.8137,46.9754,3.0343,3.2453,0.3093,2.4099,3.5491,1.1392,aparc.DKTatlas__lh__cuneus
3,sub-1000393599,aparc.DKTatlas,lh,entorhinal,737,549,2714,3.655,0.585,0.144,...,2.5989,37.5099,34.9110,3.4560,2.6710,0.1285,2.4654,2.9647,0.4993,aparc.DKTatlas__lh__entorhinal
4,sub-1000393599,aparc.DKTatlas,lh,fusiform,4115,2822,8180,2.738,0.526,0.130,...,-5.9378,39.6908,45.6286,3.9405,2.8272,0.1093,2.3304,3.1105,0.7800,aparc.DKTatlas__lh__fusiform
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13735,sub-1000393599,Yeo2011_7Networks_N1000,rh,7Networks_3,14937,9936,27688,2.611,0.492,0.123,...,-10.8846,39.2314,50.1161,4.1769,3.1173,0.3747,2.4544,4.7044,2.2500,Yeo2011_7Networks_N1000__rh__7Networks_3
13736,sub-1000393599,Yeo2011_7Networks_N1000,rh,7Networks_4,13382,9146,29555,2.909,0.582,0.127,...,-41.1954,52.2013,93.3967,3.8157,3.5262,0.9928,1.8828,5.1531,3.2703,Yeo2011_7Networks_N1000__rh__7Networks_4
13737,sub-1000393599,Yeo2011_7Networks_N1000,rh,7Networks_5,10558,7677,31072,3.196,0.792,0.143,...,-22.2837,88.8118,111.0955,3.3020,2.5300,0.3971,2.0215,4.7753,2.7538,Yeo2011_7Networks_N1000__rh__7Networks_5
13738,sub-1000393599,Yeo2011_7Networks_N1000,rh,7Networks_6,20144,13602,41999,2.696,0.641,0.135,...,-11.6287,43.5814,55.2101,3.6592,3.0563,0.5547,1.8599,4.9149,3.0550,Yeo2011_7Networks_N1000__rh__7Networks_6


In [6]:
participant_id = 1000393599
flatten_RBC_participant(participant_id).rename(columns={'value':participant_id}).set_index('measure_info').T

measure_info,aparc.DKTatlas__lh__caudalanteriorcingulate__NumVert,aparc.DKTatlas__lh__caudalmiddlefrontal__NumVert,aparc.DKTatlas__lh__cuneus__NumVert,aparc.DKTatlas__lh__entorhinal__NumVert,aparc.DKTatlas__lh__fusiform__NumVert,aparc.DKTatlas__lh__inferiorparietal__NumVert,aparc.DKTatlas__lh__inferiortemporal__NumVert,aparc.DKTatlas__lh__isthmuscingulate__NumVert,aparc.DKTatlas__lh__lateraloccipital__NumVert,aparc.DKTatlas__lh__lateralorbitofrontal__NumVert,...,Yeo2011_7Networks_N1000__lh__7Networks_6__Range_piallgi,Yeo2011_7Networks_N1000__lh__7Networks_7__Range_piallgi,Yeo2011_7Networks_N1000__rh__FreeSurfer_Defined_Medial_Wall__Range_piallgi,Yeo2011_7Networks_N1000__rh__7Networks_1__Range_piallgi,Yeo2011_7Networks_N1000__rh__7Networks_2__Range_piallgi,Yeo2011_7Networks_N1000__rh__7Networks_3__Range_piallgi,Yeo2011_7Networks_N1000__rh__7Networks_4__Range_piallgi,Yeo2011_7Networks_N1000__rh__7Networks_5__Range_piallgi,Yeo2011_7Networks_N1000__rh__7Networks_6__Range_piallgi,Yeo2011_7Networks_N1000__rh__7Networks_7__Range_piallgi
1000393599,1668,3308,4102,737,4115,7381,4828,1578,10035,4309,...,3.3341,3.4807,2.4321,1.2908,3.3515,2.25,3.2703,2.7538,3.055,2.759


In [7]:
# make_RBC_data_into_a_table([1000393599,1317462,11407866]) 

measure_info,aparc.DKTatlas__lh__caudalanteriorcingulate__NumVert,aparc.DKTatlas__lh__caudalmiddlefrontal__NumVert,aparc.DKTatlas__lh__cuneus__NumVert,aparc.DKTatlas__lh__entorhinal__NumVert,aparc.DKTatlas__lh__fusiform__NumVert,aparc.DKTatlas__lh__inferiorparietal__NumVert,aparc.DKTatlas__lh__inferiortemporal__NumVert,aparc.DKTatlas__lh__isthmuscingulate__NumVert,aparc.DKTatlas__lh__lateraloccipital__NumVert,aparc.DKTatlas__lh__lateralorbitofrontal__NumVert,...,Slab__rh__region00878__StdDev_piallgi,CC200__lh__region00047__Min_piallgi,CC400__rh__region00012__Min_piallgi,Slab__rh__region00878__Min_piallgi,CC200__lh__region00047__Max_piallgi,CC400__rh__region00012__Max_piallgi,Slab__rh__region00878__Max_piallgi,CC200__lh__region00047__Range_piallgi,CC400__rh__region00012__Range_piallgi,Slab__rh__region00878__Range_piallgi
1000393599,1668,3308,4102,737,4115,7381,4828,1578,10035,4309,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1317462,1931,4304,3388,619,4521,6401,5000,1747,8199,4887,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11407866,2014,4213,3230,787,4320,7388,6058,1574,7431,4829,...,0.0,2.4648,2.4339,3.1556,2.4648,2.4339,3.1556,0.0,0.0,0.0


In [8]:
# example_df.columns

In [9]:
example_df.isna().sum(axis=0)

subject_id        0
atlas             0
hemisphere        0
StructName        0
NumVert           0
SurfArea          0
GrayVol           0
ThickAvg          0
ThickStd          0
MeanCurv          0
GausCurv          0
FoldInd           0
CurvInd           0
Index             0
SegId             0
Mean_wgpct        0
StdDev_wgpct      0
Min_wgpct         0
Max_wgpct         0
Range_wgpct       0
SNR_wgpct         0
Mean_piallgi      0
StdDev_piallgi    0
Min_piallgi       0
Max_piallgi       0
Range_piallgi     0
feature_info      0
dtype: int64

In [10]:
# example_df['StructName'].tolist()
# example_df['hemisphere'].value_counts()

In [11]:
example_df['StructName'].value_counts().value_counts()

count
1     1535
2      619
3      322
4      259
5      208
9      203
8      198
7      198
10     196
6      188
40       1
Name: count, dtype: int64

In [12]:
count_data = example_df['StructName'].value_counts()
count_data[count_data == 40]

StructName
Background+FreeSurfer_Defined_Medial_Wall    40
Name: count, dtype: int64

In [13]:
count_data = example_df['StructName'].value_counts()
count_data[count_data == 4]

StructName
7Networks_LH_Default_Temp_13    4
inferiortemporal                4
inferiorparietal                4
7Networks_RH_SomMot_63          4
17Networks_LH_ContA_PFClv_3     4
                               ..
cuneus                          4
caudalmiddlefrontal             4
caudalanteriorcingulate         4
medialorbitofrontal             4
middletemporal                  4
Name: count, Length: 259, dtype: int64

In [14]:
count_data = example_df[['StructName','atlas']].value_counts()
count_data.value_counts()

count
1    13034
2      353
Name: count, dtype: int64

In [15]:
count_data[count_data == 2]

StructName             atlas                   
S_central              aparc.a2009s                2
S_cingul-Marginalis    aparc.a2009s                2
S_circular_insula_ant  aparc.a2009s                2
S_circular_insula_inf  aparc.a2009s                2
S_circular_insula_sup  aparc.a2009s                2
                                                  ..
Lat_Fis-ant-Vertical   aparc.a2009s                2
region00109            CC400                       2
17Networks_3           Yeo2011_17Networks_N1000    2
17Networks_2           Yeo2011_17Networks_N1000    2
17Networks_16          Yeo2011_17Networks_N1000    2
Name: count, Length: 353, dtype: int64

In [16]:
example_df['atlas'].value_counts()

atlas
Schaefer2018_1000Parcels_7Networks_order     1002
Schaefer2018_1000Parcels_17Networks_order    1002
Schaefer2018_900Parcels_17Networks_order      902
Schaefer2018_900Parcels_7Networks_order       902
Schaefer2018_800Parcels_7Networks_order       802
Schaefer2018_800Parcels_17Networks_order      802
Schaefer2018_700Parcels_7Networks_order       702
Schaefer2018_700Parcels_17Networks_order      702
Slab                                          693
Schaefer2018_600Parcels_7Networks_order       602
Schaefer2018_600Parcels_17Networks_order      602
Schaefer2018_500Parcels_7Networks_order       502
Schaefer2018_500Parcels_17Networks_order      502
Schaefer2018_400Parcels_7Networks_order       402
Schaefer2018_400Parcels_17Networks_order      402
CC400                                         376
glasser                                       362
gordon333dil                                  335
Schaefer2018_300Parcels_17Networks_order      302
Schaefer2018_300Parcels_7Networks_order     

In [17]:
example_df[example_df['atlas'] == 'Yeo2011_17Networks_N1000']

,subject_id,atlas,hemisphere,StructName,NumVert,SurfArea,GrayVol,ThickAvg,ThickStd,MeanCurv,...,Min_wgpct,Max_wgpct,Range_wgpct,SNR_wgpct,Mean_piallgi,StdDev_piallgi,Min_piallgi,Max_piallgi,Range_piallgi,feature_info
13688,sub-1000393599,Yeo2011_17Networks_N1000,lh,FreeSurfer_Defined_Medial_Wall,9543,6258,2751,0.885,1.294,0.091,...,-60.4127,96.6036,157.0164,0.3895,2.4947,0.4153,1.6581,4.1339,2.4758,Yeo2011_17Networks_N1000__lh__FreeSurfer_Defin...
13689,sub-1000393599,Yeo2011_17Networks_N1000,lh,17Networks_1,13623,8672,19547,2.116,0.646,0.139,...,-9.0185,48.8780,57.8965,2.9595,2.8081,0.2518,2.3391,3.5635,1.2245,Yeo2011_17Networks_N1000__lh__17Networks_1
13690,sub-1000393599,Yeo2011_17Networks_N1000,lh,17Networks_2,10745,7056,14383,1.983,0.578,0.132,...,-51.1054,62.3168,113.4222,2.4443,3.1184,0.2792,2.4128,3.5491,1.1363,Yeo2011_17Networks_N1000__lh__17Networks_2
13691,sub-1000393599,Yeo2011_17Networks_N1000,lh,17Networks_3,10904,7120,17942,2.320,0.635,0.118,...,-7.6208,41.5867,49.2075,2.9989,2.9659,0.5278,2.0566,4.8390,2.7824,Yeo2011_17Networks_N1000__lh__17Networks_3
13692,sub-1000393599,Yeo2011_17Networks_N1000,lh,17Networks_4,9490,6237,17974,2.685,0.588,0.120,...,-23.3178,39.6753,62.9930,3.3327,4.6257,0.5825,3.5515,5.5729,2.0214,Yeo2011_17Networks_N1000__lh__17Networks_4
13693,sub-1000393599,Yeo2011_17Networks_N1000,lh,17Networks_5,7747,5161,13358,2.447,0.487,0.124,...,-9.4500,40.4434,49.8934,3.6261,3.2079,0.2347,2.7501,3.6836,0.9335,Yeo2011_17Networks_N1000__lh__17Networks_5
13694,sub-1000393599,Yeo2011_17Networks_N1000,lh,17Networks_6,6609,4413,12045,2.557,0.476,0.123,...,-13.0442,45.1652,58.2094,4.0900,3.3344,0.3585,2.3235,4.0727,1.7492,Yeo2011_17Networks_N1000__lh__17Networks_6
13695,sub-1000393599,Yeo2011_17Networks_N1000,lh,17Networks_7,7691,5191,16432,2.913,0.641,0.122,...,-30.7357,43.1987,73.9344,3.5963,3.7517,1.3434,1.8915,5.5585,3.6670,Yeo2011_17Networks_N1000__lh__17Networks_7
13696,sub-1000393599,Yeo2011_17Networks_N1000,lh,17Networks_8,5840,3869,12818,2.892,0.629,0.122,...,-16.3872,41.3838,57.7710,3.7506,3.1185,0.9323,1.8147,5.2156,3.4009,Yeo2011_17Networks_N1000__lh__17Networks_8
13697,sub-1000393599,Yeo2011_17Networks_N1000,lh,17Networks_9,5627,4035,15647,3.081,0.746,0.139,...,-25.1028,63.3100,88.4128,2.7455,2.8458,0.5489,2.3255,5.1985,2.8730,Yeo2011_17Networks_N1000__lh__17Networks_9


In [21]:
data = load_fsdata(1000393599)
data

,subject_id,session_id,atlas,hemisphere,StructName,NumVert,SurfArea,GrayVol,ThickAvg,ThickStd,...,StdDev_wgpct,Min_wgpct,Max_wgpct,Range_wgpct,SNR_wgpct,Mean_piallgi,StdDev_piallgi,Min_piallgi,Max_piallgi,Range_piallgi
0,sub-1000393599,NaN,aparc.DKTatlas,lh,caudalanteriorcingulate,1668,1121,3493,2.870,0.588,...,5.8371,-1.8413,42.8855,44.7269,4.4281,1.9877,0.0777,1.8054,2.1455,0.3402
1,sub-1000393599,NaN,aparc.DKTatlas,lh,caudalmiddlefrontal,3308,2236,7030,2.882,0.537,...,4.6666,7.1531,40.4774,33.3243,5.0341,3.3898,0.2448,2.7003,3.8032,1.1029
2,sub-1000393599,NaN,aparc.DKTatlas,lh,cuneus,4102,2619,5753,2.019,0.490,...,5.2623,-13.1617,33.8137,46.9754,3.0343,3.2453,0.3093,2.4099,3.5491,1.1392
3,sub-1000393599,NaN,aparc.DKTatlas,lh,entorhinal,737,549,2714,3.655,0.585,...,6.0438,2.5989,37.5099,34.9110,3.4560,2.6710,0.1285,2.4654,2.9647,0.4993
4,sub-1000393599,NaN,aparc.DKTatlas,lh,fusiform,4115,2822,8180,2.738,0.526,...,5.2854,-5.9378,39.6908,45.6286,3.9405,2.8272,0.1093,2.3304,3.1105,0.7800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13735,sub-1000393599,NaN,Yeo2011_7Networks_N1000,rh,7Networks_3,14937,9936,27688,2.611,0.492,...,5.0774,-10.8846,39.2314,50.1161,4.1769,3.1173,0.3747,2.4544,4.7044,2.2500
13736,sub-1000393599,NaN,Yeo2011_7Networks_N1000,rh,7Networks_4,13382,9146,29555,2.909,0.582,...,5.8317,-41.1954,52.2013,93.3967,3.8157,3.5262,0.9928,1.8828,5.1531,3.2703
13737,sub-1000393599,NaN,Yeo2011_7Networks_N1000,rh,7Networks_5,10558,7677,31072,3.196,0.792,...,7.1063,-22.2837,88.8118,111.0955,3.3020,2.5300,0.3971,2.0215,4.7753,2.7538
13738,sub-1000393599,NaN,Yeo2011_7Networks_N1000,rh,7Networks_6,20144,13602,41999,2.696,0.641,...,6.0781,-11.6287,43.5814,55.2101,3.6592,3.0563,0.5547,1.8599,4.9149,3.0550


In [24]:
set([struct for struct in data['StructName'] if "cingulate" in struct])

{'Paracingulate_Gyrus',
 'caudalanteriorcingulate',
 'isthmuscingulate',
 'posteriorcingulate',
 'rostralanteriorcingulate'}

In [25]:
set([struct for struct in data['StructName'] if "insula" in struct])

{'G_insular_short',
 'S_circular_insula_ant',
 'S_circular_insula_inf',
 'S_circular_insula_sup',
 'insula'}

In [27]:
structs = ['Paracingulate_Gyrus', 'caudalanteriorcingulate', 'isthmuscingulate', 'posteriorcingulate', 
 'rostralanteriorcingulate', 'G_insular_short', 'S_circular_insula_ant', 'S_circular_insula_inf',
 'S_circular_insula_sup', 'insula']

In [32]:
reduced_df = data[data['StructName'].isin(structs) & (data['atlas'] == 'aparc.DKTatlas')]
reduced_df

,subject_id,session_id,atlas,hemisphere,StructName,NumVert,SurfArea,GrayVol,ThickAvg,ThickStd,...,StdDev_wgpct,Min_wgpct,Max_wgpct,Range_wgpct,SNR_wgpct,Mean_piallgi,StdDev_piallgi,Min_piallgi,Max_piallgi,Range_piallgi
0,sub-1000393599,NaN,aparc.DKTatlas,lh,caudalanteriorcingulate,1668,1121,3493,2.870,0.588,...,5.8371,-1.8413,42.8855,44.7269,4.4281,1.9877,0.0777,1.8054,2.1455,0.3402
7,sub-1000393599,NaN,aparc.DKTatlas,lh,isthmuscingulate,1578,1002,2545,2.371,0.719,...,5.6952,-1.1202,35.8699,36.9901,3.6037,2.7171,0.2753,2.1326,3.2785,1.1458
20,sub-1000393599,NaN,aparc.DKTatlas,lh,posteriorcingulate,2055,1415,4181,2.787,0.581,...,5.5395,-10.9196,38.8334,49.7531,4.2065,2.2051,0.2040,1.8125,2.6093,0.7968
23,sub-1000393599,NaN,aparc.DKTatlas,lh,rostralanteriorcingulate,1746,1153,3980,3.151,0.668,...,9.1462,-21.5070,43.9405,65.4476,2.3592,2.1292,0.0596,1.8578,2.2526,0.3948
30,sub-1000393599,NaN,aparc.DKTatlas,lh,insula,2808,1852,6365,3.252,0.735,...,6.2826,-30.7357,41.3282,72.0640,3.1291,5.0003,0.4752,2.5244,5.5729,3.0485
31,sub-1000393599,NaN,aparc.DKTatlas,rh,caudalanteriorcingulate,955,607,2092,2.914,0.668,...,7.6800,-15.6167,42.7173,58.3340,3.2970,1.9822,0.0639,1.8644,2.1991,0.3347
38,sub-1000393599,NaN,aparc.DKTatlas,rh,isthmuscingulate,1487,945,2586,2.403,0.749,...,5.6479,-3.2394,38.5529,41.7923,3.5000,3.0053,0.2426,2.4277,3.4584,1.0307
51,sub-1000393599,NaN,aparc.DKTatlas,rh,posteriorcingulate,1998,1363,4428,2.909,0.541,...,5.3520,-13.4724,37.7767,51.2491,4.3435,2.2745,0.2501,1.8760,2.9094,1.0334
54,sub-1000393599,NaN,aparc.DKTatlas,rh,rostralanteriorcingulate,1089,718,2239,2.882,0.802,...,11.2468,-59.3030,47.1047,106.4077,2.0924,2.2475,0.0629,2.0572,2.3975,0.3403
61,sub-1000393599,NaN,aparc.DKTatlas,rh,insula,2915,1976,6407,3.238,0.764,...,6.1994,-39.9900,52.2013,92.1913,3.2563,4.6736,0.5146,2.5010,5.3449,2.8439


In [34]:
# reduced_df['atlas'].value_counts()

In [37]:
reduced_df['feature_info'] = reduced_df[['atlas', 'hemisphere', 'StructName']].agg('__'.join, axis=1)

/tmp/ipykernel_128/37269277.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df['feature_info'] = reduced_df[['atlas', 'hemisphere', 'StructName']].agg('__'.join, axis=1)


In [38]:
reduced_df.columns

Index(['subject_id', 'session_id', 'atlas', 'hemisphere', 'StructName',
       'NumVert', 'SurfArea', 'GrayVol', 'ThickAvg', 'ThickStd', 'MeanCurv',
       'GausCurv', 'FoldInd', 'CurvInd', 'Index', 'SegId', 'Mean_wgpct',
       'StdDev_wgpct', 'Min_wgpct', 'Max_wgpct', 'Range_wgpct', 'SNR_wgpct',
       'Mean_piallgi', 'StdDev_piallgi', 'Min_piallgi', 'Max_piallgi',
       'Range_piallgi', 'feature_info'],
      dtype='object')

In [39]:
reduced_df[['feature_info','SurfArea', 'GrayVol']]

,feature_info,SurfArea,GrayVol
0,aparc.DKTatlas__lh__caudalanteriorcingulate,1121,3493
7,aparc.DKTatlas__lh__isthmuscingulate,1002,2545
20,aparc.DKTatlas__lh__posteriorcingulate,1415,4181
23,aparc.DKTatlas__lh__rostralanteriorcingulate,1153,3980
30,aparc.DKTatlas__lh__insula,1852,6365
31,aparc.DKTatlas__rh__caudalanteriorcingulate,607,2092
38,aparc.DKTatlas__rh__isthmuscingulate,945,2586
51,aparc.DKTatlas__rh__posteriorcingulate,1363,4428
54,aparc.DKTatlas__rh__rostralanteriorcingulate,718,2239
61,aparc.DKTatlas__rh__insula,1976,6407


In [147]:
names = reduced_df['feature_info'].apply(lambda x : x+'SurfArea').tolist() + reduced_df['feature_info'].apply(lambda x : x+'GrayVol').tolist()
names

['aparc.DKTatlas__lh__caudalanteriorcingulateSurfArea',
 'aparc.DKTatlas__lh__isthmuscingulateSurfArea',
 'aparc.DKTatlas__lh__posteriorcingulateSurfArea',
 'aparc.DKTatlas__lh__rostralanteriorcingulateSurfArea',
 'aparc.DKTatlas__lh__insulaSurfArea',
 'aparc.DKTatlas__rh__caudalanteriorcingulateSurfArea',
 'aparc.DKTatlas__rh__isthmuscingulateSurfArea',
 'aparc.DKTatlas__rh__posteriorcingulateSurfArea',
 'aparc.DKTatlas__rh__rostralanteriorcingulateSurfArea',
 'aparc.DKTatlas__rh__insulaSurfArea',
 'aparc.DKTatlas__lh__caudalanteriorcingulateGrayVol',
 'aparc.DKTatlas__lh__isthmuscingulateGrayVol',
 'aparc.DKTatlas__lh__posteriorcingulateGrayVol',
 'aparc.DKTatlas__lh__rostralanteriorcingulateGrayVol',
 'aparc.DKTatlas__lh__insulaGrayVol',
 'aparc.DKTatlas__rh__caudalanteriorcingulateGrayVol',
 'aparc.DKTatlas__rh__isthmuscingulateGrayVol',
 'aparc.DKTatlas__rh__posteriorcingulateGrayVol',
 'aparc.DKTatlas__rh__rostralanteriorcingulateGrayVol',
 'aparc.DKTatlas__rh__insulaGrayVol']

In [42]:
def load_ba1_surfarea(participant_id):
    """Loads and returns the bilateral Brodmann Area 1 surface area for a PNC
    study participant.
    """
    # First, load the subject's FreeSurfer dataframe:
    data = load_fsdata(participant_id)
    # Next, find the relevant rows:
    row_mask = (data['StructName'] == 'Brodmann.1')
    # Then extract and sum the surface areas:
    ba1_surfareas = data.loc[row_mask, 'SurfArea']
    ba1_surfarea = sum(ba1_surfareas)
    # And return this value:
    return ba1_surfarea

In [44]:
load_ba1_surfarea(1000393599)

3290

In [77]:
def load_data_of_intrest(participant_id,structs,measures):
    """Loads and returns the bilateral Brodmann Area 1 surface area for a PNC
    study participant.
    """
    # First, load the subject's FreeSurfer dataframe:
    data = load_fsdata(participant_id)
    data = data[data['atlas'] == 'aparc.DKTatlas'] # get down to just one atlas
    # Next, find the relevant rows:
    row_mask = (data['StructName'].isin(structs))
    # print(sum(row_mask))
    # Then extract the data:
    output_data = data.loc[row_mask, measures].values.flatten().tolist()
    # And return this value:
    return output_data

In [78]:
structs = ['Paracingulate_Gyrus', 'caudalanteriorcingulate', 'isthmuscingulate', 'posteriorcingulate', 
     'rostralanteriorcingulate', 'G_insular_short', 'S_circular_insula_ant', 'S_circular_insula_inf',
     'S_circular_insula_sup', 'insula']
measures = ['SurfArea','GrayVol']
load_data_of_intrest(1000393599,structs,measures)

[1121,
 3493,
 1002,
 2545,
 1415,
 4181,
 1153,
 3980,
 1852,
 6365,
 607,
 2092,
 945,
 2586,
 1363,
 4428,
 718,
 2239,
 1976,
 6407]

In [80]:
# Participant meta-data is generally located in the BIDS repository for each
# study:
rbcdata_path = Path('/home/jovyan/shared/data/RBC')
train_filepath = rbcdata_path / 'train_participants.tsv'
test_filepath = rbcdata_path / 'test_participants.tsv'

# Load the PNC participants TSV files...
with train_filepath.open('r') as f:
    train_data = pd.read_csv(f, sep='\t')
with test_filepath.open('r') as f:
    test_data = pd.read_csv(f, sep='\t')

# We can also concatenate the two datasets into a single dataset of all
# study participants:
all_data = pd.concat([train_data, test_data])

# Display the full dataframe:
all_data

,participant_id,study,study_site,session_id,wave,age,sex,race,ethnicity,bmi,handedness,participant_education,parent_1_education,parent_2_education,p_factor,internalizing_mcelroy_harmonized_all_samples,externalizing_mcelroy_harmonized_all_samples,attention_mcelroy_harmonized_all_samples,cubids_acquisition_group
0,1000393599,PNC,PNC1,PNC1,1,15.583333,Male,Black,not Hispanic or Latino,22.15,Right,9th Grade,Complete primary,Complete secondary,0.589907,-0.449373,-0.630780,-1.842178,1
1,1001970838,PNC,PNC1,PNC1,1,17.833333,Male,Other,Hispanic or Latino,23.98,Right,11th Grade,Complete tertiary,Complete tertiary,-0.659061,0.531072,0.392751,0.190706,1
2,1007995238,PNC,PNC1,PNC1,1,13.750000,Female,Other,not Hispanic or Latino,23.77,Right,6th Grade,Complete tertiary,Complete primary,-1.608375,-0.744118,-0.314187,-0.432662,1
3,1011497669,PNC,PNC1,PNC1,1,16.666667,Male,White,not Hispanic or Latino,29.68,Right,9th Grade,Complete tertiary,Complete tertiary,-1.233807,-0.896835,-0.449099,0.111167,1
4,1017092387,PNC,PNC1,PNC1,1,18.666667,Female,Black,not Hispanic or Latino,23.24,Right,11th Grade,Complete primary,Complete primary,-0.923100,-0.313455,2.204168,-0.782266,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,969649154,PNC,PNC1,PNC1,1,12.333333,Male,White,not Hispanic or Latino,17.38,Right,5th Grade,Complete tertiary,Complete secondary,NaN,-0.148520,0.556444,0.024228,1
530,970890500,PNC,PNC1,PNC1,1,18.166667,Female,White,not Hispanic or Latino,30.89,Right,11th Grade,Complete secondary,Complete secondary,NaN,0.993806,1.578177,-0.373470,1
531,975856179,PNC,PNC1,PNC1,1,11.000000,Male,White,not Hispanic or Latino,15.67,Right,4th Grade,Complete primary,Complete secondary,NaN,-1.026645,-0.582212,1.333857,1
532,984757368,PNC,PNC1,PNC1,1,13.416667,Male,Black,not Hispanic or Latino,16.66,Right,5th Grade,Complete primary,NaN,NaN,0.360029,-0.515655,1.509584,114


In [88]:
# First load in surface area data for each participant:
print("Loading data...")     

# We will put the rows in this dictionary of lists as we build the dataframe:
all_vars = {
    'participant_id': [],
    'ba1_surface_area': [],
    'p_factor': []}

# We'll display a progress bar `prog` as we go also:
from ipywidgets import IntProgress
prog = IntProgress(min=0, max=len(all_data))
display(prog)

# Okay, loop through each row of the `all_data` dataframe, which contains both
# training and test subjects, load their BA1 data, and store it in the
# all_vars dictionary.
for (ii, row) in all_data.iterrows():
    # Extract the participant ID and p_factor (which will be NaN for test
    # participants).
    participant_id = row['participant_id']
    p_factor = row['p_factor']
    # Load the surface area for this participant:
    try:
        surf_area = load_data_of_intrest(participant_id,structs,measures)
    except FileNotFoundError:
        # Some subjects are just missing the file, so we code them as NaN.
        surf_area = np.nan
    # Append the participant ID and their surface area to our dataset:
    all_vars['participant_id'].append(participant_id)
    all_vars['ba1_surface_area'].append(surf_area)
    all_vars['p_factor'].append(p_factor)
    # Increment the progress bar counter:
    prog.value += 1

# Convert train_vars into a dataframe.
all_vars = pd.DataFrame(all_vars)

# Extract the training and test subjects into separate dataframes; the test
# participants can be identified as those having NaN values for their
# p_factor column.
train_vars = all_vars[~np.isnan(all_vars['p_factor'])]
test_vars = all_vars[np.isnan(all_vars['p_factor'])]

# Display the finished dataframe.
all_vars

Loading data...


IntProgress(value=0, max=1601)

,participant_id,ba1_surface_area,p_factor
0,1000393599,"[1121, 3493, 1002, 2545, 1415, 4181, 1153, 398...",0.589907
1,1001970838,"[884, 2497, 935, 2385, 1174, 3343, 949, 2737, ...",-0.659061
2,1007995238,"[1117, 3358, 978, 2626, 1291, 3778, 1145, 3497...",-1.608375
3,1011497669,"[1027, 3196, 1131, 3384, 1351, 3958, 1037, 353...",-1.233807
4,1017092387,"[1048, 3148, 1146, 2853, 1467, 4180, 1097, 355...",-0.923100
...,...,...,...
1596,969649154,"[1173, 3497, 1131, 3007, 1412, 3933, 1271, 409...",NaN
1597,970890500,"[740, 2474, 752, 2185, 977, 2860, 709, 2542, 1...",NaN
1598,975856179,"[1205, 3815, 1138, 3525, 1532, 4692, 1251, 424...",NaN
1599,984757368,"[1119, 3468, 987, 2799, 1205, 3520, 1388, 3970...",NaN


In [102]:
all_vars.to_csv("/home/jovyan/Projects/Neurohack_Group8_Miniproject/flattened_data/all.csv")

In [120]:
all_vars[all_vars['ba1_surface_area'].apply(lambda x: isinstance(x, list))]

,participant_id,ba1_surface_area,p_factor
0,1000393599,"[1121, 3493, 1002, 2545, 1415, 4181, 1153, 398...",0.589907
1,1001970838,"[884, 2497, 935, 2385, 1174, 3343, 949, 2737, ...",-0.659061
2,1007995238,"[1117, 3358, 978, 2626, 1291, 3778, 1145, 3497...",-1.608375
3,1011497669,"[1027, 3196, 1131, 3384, 1351, 3958, 1037, 353...",-1.233807
4,1017092387,"[1048, 3148, 1146, 2853, 1467, 4180, 1097, 355...",-0.923100
...,...,...,...
1596,969649154,"[1173, 3497, 1131, 3007, 1412, 3933, 1271, 409...",NaN
1597,970890500,"[740, 2474, 752, 2185, 977, 2860, 709, 2542, 1...",NaN
1598,975856179,"[1205, 3815, 1138, 3525, 1532, 4692, 1251, 424...",NaN
1599,984757368,"[1119, 3468, 987, 2799, 1205, 3520, 1388, 3970...",NaN


In [155]:

# all_good_vars = all_vars[all_vars['ba1_surface_area'].apply(lambda x: isinstance(x, list))]
# #all_good_vars.set_index('participant_id')
# print(all_good_vars.shape)

# expanded = pd.DataFrame(all_good_vars['ba1_surface_area'].tolist(), columns=names)
# print(expanded.shape)
# expanded_data = pd.concat([all_good_vars.drop(columns=['ba1_surface_area']), expanded], axis=1)
# print(expanded_data.shape)

# expanded_data

(1592, 3)
(1592, 20)
(1601, 22)


,participant_id,p_factor,aparc.DKTatlas__lh__caudalanteriorcingulateSurfArea,aparc.DKTatlas__lh__isthmuscingulateSurfArea,aparc.DKTatlas__lh__posteriorcingulateSurfArea,aparc.DKTatlas__lh__rostralanteriorcingulateSurfArea,aparc.DKTatlas__lh__insulaSurfArea,aparc.DKTatlas__rh__caudalanteriorcingulateSurfArea,aparc.DKTatlas__rh__isthmuscingulateSurfArea,aparc.DKTatlas__rh__posteriorcingulateSurfArea,...,aparc.DKTatlas__lh__caudalanteriorcingulateGrayVol,aparc.DKTatlas__lh__isthmuscingulateGrayVol,aparc.DKTatlas__lh__posteriorcingulateGrayVol,aparc.DKTatlas__lh__rostralanteriorcingulateGrayVol,aparc.DKTatlas__lh__insulaGrayVol,aparc.DKTatlas__rh__caudalanteriorcingulateGrayVol,aparc.DKTatlas__rh__isthmuscingulateGrayVol,aparc.DKTatlas__rh__posteriorcingulateGrayVol,aparc.DKTatlas__rh__rostralanteriorcingulateGrayVol,aparc.DKTatlas__rh__insulaGrayVol
0,1.000394e+09,0.589907,1121.0,3493.0,1002.0,2545.0,1415.0,4181.0,1153.0,3980.0,...,607.0,2092.0,945.0,2586.0,1363.0,4428.0,718.0,2239.0,1976.0,6407.0
1,1.001971e+09,-0.659061,884.0,2497.0,935.0,2385.0,1174.0,3343.0,949.0,2737.0,...,710.0,1892.0,829.0,2187.0,1150.0,3410.0,849.0,2656.0,2214.0,7093.0
2,1.007995e+09,-1.608375,1117.0,3358.0,978.0,2626.0,1291.0,3778.0,1145.0,3497.0,...,677.0,2119.0,824.0,2325.0,1326.0,4015.0,886.0,2765.0,1843.0,6499.0
3,1.011498e+09,-1.233807,1027.0,3196.0,1131.0,3384.0,1351.0,3958.0,1037.0,3538.0,...,927.0,2710.0,926.0,2629.0,1522.0,4383.0,927.0,2976.0,2014.0,7053.0
4,1.017092e+09,-0.923100,1048.0,3148.0,1146.0,2853.0,1467.0,4180.0,1097.0,3550.0,...,757.0,2742.0,905.0,2533.0,1254.0,3735.0,686.0,2560.0,1932.0,6429.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,NaN,NaN,1063.0,2900.0,1453.0,4516.0,1405.0,3764.0,1262.0,3679.0,...,608.0,1659.0,1212.0,3622.0,1366.0,4206.0,635.0,1952.0,2443.0,7617.0
874,NaN,NaN,1190.0,3755.0,861.0,2222.0,1193.0,3353.0,1291.0,4262.0,...,853.0,2619.0,912.0,2471.0,1386.0,4121.0,979.0,3110.0,2145.0,6812.0
923,NaN,NaN,1083.0,3845.0,996.0,2815.0,1265.0,3727.0,1209.0,3806.0,...,638.0,2155.0,808.0,2451.0,1083.0,3462.0,795.0,2462.0,2220.0,8157.0
1509,NaN,NaN,1129.0,3609.0,1079.0,3384.0,1329.0,4220.0,1226.0,4138.0,...,893.0,2967.0,933.0,2904.0,1439.0,4313.0,768.0,2756.0,2066.0,6672.0


In [159]:
# Keep only rows where 'ba1_surface_area' is a list
all_good_vars = all_vars[all_vars['ba1_surface_area'].apply(lambda x: isinstance(x, list))].copy()

# (Optional) Reset index or set participant_id as index if needed
# all_good_vars = all_good_vars.set_index('participant_id')  # Uncomment if desired

# Show shape of filtered dataframe
print("Filtered shape:", all_good_vars.shape)

# Expand list column into separate columns
expanded = pd.DataFrame(all_good_vars['ba1_surface_area'].tolist(), columns=names)

print("Expanded shape:", expanded.shape)

# Reset index before combining
all_good_vars = all_good_vars.reset_index(drop=True)
expanded = expanded.reset_index(drop=True)

# Now concatenate
expanded_data = pd.concat([all_good_vars.drop(columns=['ba1_surface_area']), expanded], axis=1)
print("Final shape:", expanded_data.shape)

# Save it
expanded_data.to_csv('/home/jovyan/Projects/Neurohack_Group8_Miniproject/flattened_data/all_cleaned.csv')

Filtered shape: (1592, 3)
Expanded shape: (1592, 20)
Final shape: (1592, 22)


In [130]:
all_vars['ba1_surface_area'].apply(type).value_counts()

ba1_surface_area
<class 'list'>     1592
<class 'float'>       9
Name: count, dtype: int64